# Test of whether LanguageModelLoader deliver batch where each row is a continuous set of tokens


Status is that the test works with

-MyLanguageModelLoader 

-fastai's LanguageModelLoader, however, throws an exception while indexing the jagged array


In [7]:
%reload_ext autoreload
%autoreload 2

In [8]:
from fastai.text import * 
import numpy as np
from languagemodelloader import *

In [9]:
#create test data so that we can control whether the LanguageModelLoader returns contigous tokens 
#set b_ptt to for the current verion of fastai' LanguageModelLoader and 0 for MyLanguageModelLoader
#p_bptt    = .95 current verion of fastai' LanguageModelLoader 
p_bptt     =  0  #deactivates changes in the sequence length in MyLanguageModelLoader so that we do not wrap around
seq_len    =  9  #ie bptt
bs         =  4
iterations = 11
minTokens  =  4

jagged = []
countTokens = 0
totalTokens = bs*seq_len*iterations
print(f"totalTokens:{totalTokens}")
rand_interval=20
while countTokens < totalTokens:
    nb = totalTokens-countTokens if   totalTokens-countTokens <rand_interval\
                                 else minTokens+int(np.random.random() * rand_interval)
    jagged.append(np.arange(countTokens+1,countTokens+1+nb))
    countTokens = jagged[-1][-1]
jagged = np.asarray(jagged)    
print(f"countTokens:{countTokens}")
for j in jagged: print(j)

totalTokens:396
countTokens:396
[1 2 3 4 5 6 7]
[ 8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26]
[27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
[48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66]
[67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84]
[ 85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101]
[102 103 104 105 106 107 108 109 110 111 112 113 114]
[115 116 117 118 119]
[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137]
[138 139 140 141 142 143 144 145 146 147 148]
[149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169]
[170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191]
[192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210]
[211 212 213 214 215 216 217 218 219 220 221 222 223 224]
[225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242]
[243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 

In [10]:
trainIDS = validIDS = jagged
db   = TextLMDataBunch.from_ids( ".", None, trainIDS, validIDS, bptt=seq_len, p_bptt=p_bptt, bs=bs)
data = MyLanguageModelLoader(dataset=db.train_dl, bptt=seq_len, p_bptt=p_bptt, bs=bs, shuffle=False)
            
def getAllBatches(data,epochs=1):
    x=None
    for i in range(epochs):
        for xb,yb in data:
            if x is None: x = xb[:,0:seq_len].data.numpy()
            else:         x = np.concatenate((x, xb[:,0:seq_len].data.numpy()),axis=1)
            continue
    return x            
batches = getAllBatches(data)

In [11]:
#The input was size to an batches that do not wrap aoround. Also the input contains continuously 
#increasing numbers. Therefore the diff below from one to the next column must be 1 for a properly aligned batches
b_diff = batches[:,1:] - batches[:,:-1]

In [12]:
assert b_diff.flatten().all()